## Simple AI Agent using the LangGraph pre-built create_react_agent

See https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent 

In [1]:
%pip install langchain-core langgraph>0.2.27 langchain langchain-openai langchain-community python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

True

In [3]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("gpt-4o-mini", model_provider="openai")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

Query not requiring any tools

In [4]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream( {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How's it going in San Francisco?


Query requiring tool use.

In [5]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_R0y7OgdHSQARZi1HdUe35uzM)
 Call ID: call_R0y7OgdHSQARZi1HdUe35uzM
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Monday, March 31, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/march/2025-03-31", "content": "San Francisco, California Weather: Monday, March 31, 2025. Rainy weather, precipitation in the form of water droplets falling from the sky.", "score": 0.9505628}, {"title": "San Francisco weather in March 2025 | California, USA", "url": "https://www.weather2travel.com/california/san-francisco/march/", "content": "San Francisco weather in March 2025 | California